In [1]:
%%configure -f
{
    "driverMemory": "32G", 
    "executorCores": 2, 
    "numExecutors": 10, 
    "executorMemory": "16G",
    "kind": "pyspark3",
    "conf":{
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [2]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
145,application_1690391939543_0146,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-145>

In [3]:
bucket_name = "<bucket-name>"
dataset_path = "s3://{}/tmp/Accidente.csv".format(bucket_name)
dataset_path_2 = "s3://{}/tmp/ia_tp1/accidentes-dummies-fc-windchill-precipitation-windspeed-pred.csv".format(bucket_name)
output_path = "s3://{}/tmp/ia_tp1/".format(bucket_name)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from datetime import datetime
import math
from pyspark.sql.types import (
    StringType,
    ArrayType,
    StructType,
    StructField,
    BooleanType,
    IntegerType,
    FloatType,
)
from pyspark.sql import functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df = spark.read.csv(dataset_path, header=True)
df = df.filter(F.col("City").isNotNull())
df = df.filter(F.col("ZipCode").isNotNull())
df = df.filter(F.col("Timezone").isNotNull())
df = df.filter(F.col("Airport_Code").isNotNull())
df = df.filter(F.col("Weather_Timestamp").isNotNull())
df = df.filter(F.col("Temperature(F)").isNotNull())
df = df.filter(F.col("Humidity(%)").isNotNull())
df = df.filter(F.col("Pressure(in)").isNotNull())
df = df.filter(F.col("Visibility(mi)").isNotNull())
df = df.filter(F.col("Wind_Direction").isNotNull())
df = df.filter(F.col("Weather_Condition").isNotNull())
df = df.filter(F.col("Sunrise_Sunset").isNotNull())
df = df.filter(F.col("Civil_Twilight").isNotNull())
df = df.filter(F.col("Nautical_Twilight").isNotNull())
df = df.filter(F.col("Astronomical_Twilight").isNotNull())
df = df.drop_duplicates()

df.createOrReplaceTempView("accidentes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
states_to_region_csv = """
al,southeast
ar,southeast
az,southwest
ca,west
co,west
ct,northeast
dc,southeast
de,southeast
fl,southeast
ga,southeast
ia,midwest
id,west
il,midwest
in,midwest
ks,midwest
ky,southeast
la,southeast
ma,northeast
md,northeast
me,northeast
mi,midwest
mn,midwest
mo,midwest
ms,southeast
mt,west
nc,southeast
nd,midwest
ne,midwest
nh,northeast
nj,northeast
nm,southwest
nv,west
ny,northeast
oh,midwest
ok,southwest
or,west
pa,northeast
ri,northeast
sc,southeast
sd,midwest
tn,southeast
tx,southwest
ut,west
va,southeast
vt,northeast
wa,west
wi,midwest
wv,southeast
wy,west
"""

state_to_region_map = {v.split(",")[0]: v.split(",")[1] for v in states_to_region_csv.split("\n") if len(v) != 0}

weathercondition_to_weathertype_csv = """
blowing snow,snowy
ice pellets,sleet
clear,sunny
fog,restricted_visibility
heavy ice pellets,sleet
mist,restricted_visibility
funnel cloud,windy
heavy drizzle,rainy
rain shower,rainy
a precipitation,rainy
squalls,windy
light sleet,sleet
heavy sleet,sleet
snow and thunder,snowy
haze,restricted_visibility
light haze,restricted_visibility
patches of fog,restricted_visibility
light snow showers,snowy
light blowing snow,snowy
thunder and hail,snowy
widespread dust,restricted_visibility
windy,windy
light freezing drizzle,rainy
thunderstorms and snow,snowy
heavy blowing snow,snowy
heavy freezing drizzle,snowy
light freezing rain,rainy
heavy rain showers,rainy
sleet,sleet
thunderstorms and rain,rainy
light freezing fog,restricted_visibility
sand,restricted_visibility
showers in the vicinity,rainy
light rain shower,rainy
rain,rainy
snow and sleet,snowy
light rain,rainy
overcast,cloudy
light thunderstorms and rain,rainy
partial fog,restricted_visibility
mostly cloudy,cloudy
tornado,windy
light thunderstorm,windy
light rain with thunder,windy
drizzle,rainy
heavy smoke,restricted_visibility
shallow fog,restricted_visibility
light snow,snowy
heavy t-storm,rainy
freezing rain,rainy
heavy snow,snowy
t-storm,rainy
thunder,cloudy
light rain showers,rainy
snow,snowy
light snow and sleet,sleet
snow showers,snowy
heavy snow with thunder,snowy
light snow with thunder,snowy
hail,snowy
partly cloudy,cloudy
smoke,restricted_visibility
heavy thunderstorms and rain,rainy
heavy freezing rain,rainy
dust whirlwinds,restricted_visibility
heavy thunderstorms and snow,snowy
n,sunny
heavy rain,rainy
light snow shower,snowy
rain showers,rainy
light thunderstorms and snow,snowy
thunder in the vicinity,cloudy
low drifting snow,snowy
scattered clouds,cloudy
snow grains,snowy
light drizzle,rainy
wintry mix,rainy
light snow grains,snowy
heavy thunderstorms with small hail,windy
fair,sunny
light hail,snowy
volcanic ash,restricted_visibility
light ice pellets,sleet
dust whirls,restricted_visibility
light fog,restricted_visibility
cloudy,cloudy
thunderstorm,windy
blowing dust,restricted_visibility
blowing sand,restricted_visibility
small hail,snowy
drizzle and fog,rainy
"""

environment_influencers = {
    "Amenity": 0,
    "Bump": 0,
    "Crossing": 0,
    "Give_Way": 0,
    "Junction": 0,
    "No_Exit": 0,
    "Railway": 0,
    "Roundabout": 0,
    "Station": 0,
    "Stop": 0,
    "Traffic_Calming": 0,
    "Traffic_Signal": 0,
    "Turning_Loop": 0,
}

environment_influencers = {k:(1/len(environment_influencers)) for k, _ in environment_influencers.items()}

weathercondition_to_weathertype_map = {v.split(",")[0]: v.split(",")[1] for v in weathercondition_to_weathertype_csv.split("\n") if len(v) != 0}

@F.udf(returnType=StringType())
def set_region_by_state(state):
    return state_to_region_map[state.lower()]


@F.udf(returnType=ArrayType(StringType()))
def set_weather_type_by_weather_condition(weather_condition):
    return [weathercondition_to_weathertype_map[v.strip().lower()] for v in weather_condition.strip().split("/")]


@F.udf(returnType=StringType())
def get_season(start_time):
    datetime_object = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    month_and_day = int("{}{:02d}".format(datetime_object.month, datetime_object.day))
    if month_and_day <= 319:
        season = "winter"
    elif month_and_day <= 621:
        season = "spring"
    elif month_and_day <= 923:
        season = "summer"
    elif month_and_day <= 1221:
        season = "fall"
    else:
        season = "winter"
    return season


@F.udf(returnType=StringType())
def get_quarter(start_time):
    datetime_object = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    month_and_day = int("{}{:02d}".format(datetime_object.month, datetime_object.day))
    if month_and_day <= 331:
        quarter = "q1"
    elif month_and_day <= 630:
        quarter = "q2"
    elif month_and_day <= 930:
        quarter = "q3"
    elif month_and_day <= 1231:
        quarter = "q4"
    return quarter


@F.udf(returnType=StringType())
def get_day_part(start_time):
    datetime_object = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    hour = datetime_object.hour
    if hour < 4:
        day_part = "night"
    elif hour < 12:
        day_part = "morning"
    elif hour < 17:
        day_part = "afternoon"
    elif hour < 21:
        day_part = "evening" 
    else:
        day_part = "night"
    return day_part


@F.udf(returnType=StringType())
def get_wind_scale(wind_speed):
    if wind_speed is None:
        return "calm"
    
    wind_speed = float(wind_speed) * 1.6093 # mph a km/h
    if wind_speed < 1:
        scale = "calm"
    elif wind_speed < 3:
        scale = "light air"
    elif wind_speed < 7:
        scale = "light breeze"
    elif wind_speed < 12:
        scale = "gentle breeze"
    elif wind_speed < 18:
        scale = "moderate breeze"
    elif wind_speed < 24:
        scale = "fresh breeze"
    elif wind_speed < 31:
        scale = "strong breeze"
    elif wind_speed < 38:
        scale = "near gale"
    elif wind_speed < 46:
        scale = "gale"
    elif wind_speed < 54:
        scale = "severe gale"
    elif wind_speed < 63:
        scale = "storm"
    elif wind_speed < 72:
        scale = "violent storm"
    elif wind_speed < 83:
        scale = "hurricane"
    else:
        scale = "hurricane"
    return scale


@F.udf(returnType=IntegerType())
def get_datetime_part(column_value, part_name):
    return getattr(datetime.strptime(column_value, '%Y-%m-%d %H:%M:%S'), part_name)


suffix_replace = lambda suffix: suffix.strip().lower().replace(" ", "_")
def as_column_name(col_name, suffix):
    return col_name.lower() + "__" + suffix.strip().lower().replace(" ", "_")


@F.udf(returnType=IntegerType())
def assing_value(column_name, new_column_name, column_value):
    result = False
    if column_value is None:
        return int(result)
    
    if new_column_name.startswith("weather_type"):
        result = any(new_column_name == as_column_name(column_name, v) for v in column_value)
    else:
        result = new_column_name == as_column_name(column_name, column_value)
    
    return int(result)


@F.udf(returnType=FloatType())
def get_coordinate(latitude, longitude, coordinate_name):
    latitude = float(latitude)
    longitude = float(longitude)
    if coordinate_name == "x":
        return math.cos(latitude) * math.cos(longitude)
    elif coordinate_name == "y":
        return math.cos(latitude) * math.sin(longitude)
    elif coordinate_name == "z":
        return math.sin(latitude)
    
    
@F.udf(returnType=FloatType())
# @F.udf(returnType=ArrayType(FloatType()))
def determine_environment_influence_degree(
    amenity, bump, crossing, give_Way, 
    junction, no_Exit, railway, roundabout, 
    station, stop, traffic_calming, 
    traffic_signal, turning_loop
):
    columns_values = [
        amenity, bump, crossing, give_Way, 
        junction, no_Exit, railway, roundabout, 
        station, stop, traffic_calming, 
        traffic_signal, turning_loop
    ]
    columns = [int(v) for v in columns_values]
    weights = list(environment_influencers.values())
    weights_by_columns = [weights[i] for i, v in enumerate(columns) if v == 1]
    degree = sum(weights_by_columns)
    return round(degree, 6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df = df.withColumn("Region", set_region_by_state(F.col("State")))
df = df.withColumn("Season", get_season(F.col("Start_Time")))
df = df.withColumn("Quarter", get_quarter(F.col("Start_Time")))
df = df.withColumn("Day_Part", get_day_part(F.col("Start_Time")))
df = df.withColumn("Wind_Scale", get_wind_scale(F.col("Wind_Speed(mph)")))
df = df.withColumn("Weather_Type", set_weather_type_by_weather_condition(F.col("Weather_Condition")))
df = df.withColumn("Start_x", get_coordinate(F.col("Start_Lat"), F.col("Start_Lng"), F.lit("x")))
df = df.withColumn("Start_y", get_coordinate(F.col("Start_Lat"), F.col("Start_Lng"), F.lit("y")))
df = df.withColumn("Start_z", get_coordinate(F.col("Start_Lat"), F.col("Start_Lng"), F.lit("z")))
df = df.withColumn(
    "Environment_Influence", 
    determine_environment_influence_degree(
        F.col("Amenity").cast(BooleanType()), F.col("Bump").cast(BooleanType()), F.col("Crossing").cast(BooleanType()), 
        F.col("Give_Way").cast(BooleanType()), F.col("Junction").cast(BooleanType()), F.col("No_Exit").cast(BooleanType()), 
        F.col("Railway").cast(BooleanType()), F.col("Roundabout").cast(BooleanType()), F.col("Station").cast(BooleanType()), 
        F.col("Stop").cast(BooleanType()), F.col("Traffic_Calming").cast(BooleanType()), F.col("Traffic_Signal").cast(BooleanType()), 
        F.col("Turning_Loop").cast(BooleanType())
    )
)
df = df.fillna({"Environment_Influence": 0.0})

df = (df
      .withColumnRenamed("Distance(mi)", "Distance")
      .withColumnRenamed("Temperature(F)", "Temperature")
      .withColumnRenamed("Wind_Chill(F)", "Wind_Chill")
      .withColumnRenamed("Humidity(%)", "Humidity")
      .withColumnRenamed("Pressure(in)", "Pressure")
      .withColumnRenamed("Visibility(mi)", "Visibility")
      .withColumnRenamed("Wind_Speed(mph)", "Wind_Speed")
      .withColumnRenamed("Precipitation(in)", "Precipitation")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df.write.parquet(output_path + "accidentes_new_attributes", mode="overwrite", compression="snappy")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df = spark.read.parquet(output_path + "accidentes_new_attributes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
unique_values = {}
columns_to_check = [
    "Side",
    "City",
    "County",
    "State",
    "Airport_Code",
    "Wind_Direction",
    "Weather_Condition",
    "Amenity",
    "Bump",
    "Crossing",
    "Give_Way",
    "Junction",
    "No_Exit",
    "Railway",
    "Roundabout",
    "Station",
    "Stop",
    "Traffic_Calming",
    "Traffic_Signal",
    "Turning_Loop",
    "Sunrise_Sunset",
    "Civil_Twilight",
    "Nautical_Twilight",
    "Astronomical_Twilight",
    "Region",
    "Season",
    "Quarter",
    "Day_Part",
    "Wind_Scale",
    "Weather_Type",
]

df.createOrReplaceTempView("accidentes")
for col_name in columns_to_check:
    if col_name == "Weather_Type":
        values = list(set(weathercondition_to_weathertype_map.values()))
    else:
        values = [row[col_name] for row in spark.sql("select distinct {} from accidentes".format(col_name)).collect()]
        
    unique_values[col_name] = set([str(v).lower() for v in values if v is not None and len(str(v).strip()) > 0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
weather_condition = set(unique_values["Weather_Condition"])
new_values = set()
for v in weather_condition:
    values = v.strip().split("/")
    for new_value in values:
        new_values.add(new_value.strip())

unique_values["Weather_Condition_old"] = set(unique_values["Weather_Condition"])
unique_values["Weather_Condition"] = new_values

wind_direction = set(unique_values["Wind_Direction"])
new_values = {}
for v in wind_direction:
    if v == "e":
        new_values[v] = "east"
    elif v == "n":
        new_values[v] = "north"
    elif v == "s":
        new_values[v] = "south"
    elif v == "w":
        new_values[v] = "west"
    elif v == "var":
        new_values[v] = "variable"
    else:
        new_values[v] = v

unique_values["Wind_Direction_old"] = set(unique_values["Wind_Direction"])
unique_values["Wind_Direction"] = new_values
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
try:
    del unique_values["Weather_Condition_old"]
except KeyError:
    pass

try:
    del unique_values["Weather_Condition"]
except KeyError:
    pass

try:
    del unique_values["Wind_Direction_old"]
except KeyError:
    pass

try:
    del unique_values["City"]
except KeyError:
    pass

try:
    del unique_values["County"]
except KeyError:
    pass

try:
    del unique_values["State"]
except KeyError:
    pass

try:
    del unique_values["Airport_Code"]
except KeyError:
    pass

two_values_only = []
for k, v in unique_values.items():
    if len(v) <= 2:
        two_values_only.append(k)
        
for k in two_values_only:
    del unique_values[k]

# City,11639
# County,1692
# State,49
# Airport_Code,1995
# Street, 160715

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
for key, value in unique_values.items():
    print("{},{}".format(key, len(value)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Wind_Direction,23
Region,5
Season,4
Quarter,4
Day_Part,4
Wind_Scale,13
Weather_Type,7

In [31]:
df = df.withColumn("start_time__year", get_datetime_part(F.col("Start_Time"), F.lit("year")))
df = df.withColumn("start_time__month", get_datetime_part(F.col("Start_Time"), F.lit("month")))
df = df.withColumn("start_time__day", get_datetime_part(F.col("Start_Time"), F.lit("day")))
df = df.withColumn("start_time__hour", get_datetime_part(F.col("Start_Time"), F.lit("hour")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
def get_new_column_names(col_name):
    names = []
    if isinstance(unique_values[col_name], set):
        names = [as_column_name(col_name, v) for v in unique_values[col_name]]
    elif isinstance(unique_values[col_name], dict):
        names = [as_column_name(col_name, v) for v in set(unique_values[col_name].values())]
    return names

# column_names = ["State"]
# for column_name in column_names:
for column_name in unique_values.keys():
    for new_column_name in get_new_column_names(column_name):
        df = df.withColumn(new_column_name, assing_value(F.lit(column_name), F.lit(new_column_name), F.col(column_name)))

df = df.drop(*[
    "ID",
    "TMC",
    "Source",
    "Start_Time",
    "End_Time",
    "Start_Lat",
    "Start_Lng",
    "End_Lat",
    "End_Lng",
    "Description",
    "Number",
    "Side",
    "Street",
    "City",
    "County",
    "State",
    "Zipcode",
    "Country",
    "TimeZone",
    "Airport_Code",
    "Weather_Timestamp",
    "Wind_Direction",
    "Weather_Condition",
    "Amenity",
    "Bump",
    "Crossing",
    "Give_Way",
    "Junction",
    "No_Exit",
    "Railway",
    "Roundabout",
    "Station",
    "Stop",
    "Traffic_Calming",
    "Traffic_Signal",
    "Turning_Loop",
    "Sunrise_Sunset",
    "Civil_Twilight",
    "Nautical_Twilight",
    "Astronomical_Twilight",
    "Weather_Type",
])

df.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+-----------+----------+--------+--------+----------+----------+-------------+---------+------+-------+---------+---------------+-----------+-----------+-----------+---------------------+----------------+-----------------+---------------+----------------+--------------------+------------------+------------------+------------------------+-------------------+-------------------+-------------------+---------------------+------------------+-------------------+--------------------+-------------------+-------------------+------------------+---------------------+-------------------+-------------------+--------------------+------------+-----------------+-----------------+-----------------+---------------+--------------+--------------+--------------+------------+-----------+-----------+-----------+-----------+-----------------+-------------------+-----------------+---------------+-------------------------+-------------------------+---------------------+------------------------

In [33]:
df.write.parquet(output_path + "accidentes_dummies", mode="overwrite", compression="snappy")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df = spark.read.parquet(output_path + "accidentes_dummies")
df.repartition(1).write.csv(output_path + "accidentes_dummies_onefile", header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df = spark.read.parquet(output_path + "accidentes_dummies")
df.createOrReplaceTempView("accidentes_dummies")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
spark.sql("select weather_type__cloudy, weather_type__sunny, weather_type__restricted_visibility, weather_type__rainy, weather_type__snowy, weather_type__windy, weather_type__sleet from accidentes_dummies where weather_type__sunny != 0").show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-----------------------------------+-------------------+-------------------+-------------------+-------------------+
|weather_type__cloudy|weather_type__sunny|weather_type__restricted_visibility|weather_type__rainy|weather_type__snowy|weather_type__windy|weather_type__sleet|
+--------------------+-------------------+-----------------------------------+-------------------+-------------------+-------------------+-------------------+
|0                   |1                  |0                                  |0                  |0                  |0                  |0                  |
|0                   |1                  |0                                  |0                  |0                  |0                  |0                  |
|0                   |1                  |0                                  |0                  |0                  |0                  |0                  |
|0                   |1                  |0   

In [8]:
spark.sql("select start_time__year, severity, count(0) from accidentes_dummies group by start_time__year, severity order by start_time__year, severity").show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------+--------+
|start_time__year|severity|count(0)|
+----------------+--------+--------+
|2016            |1       |219     |
|2016            |2       |261987  |
|2016            |3       |121901  |
|2016            |4       |13664   |
|2017            |1       |270     |
|2017            |2       |448322  |
|2017            |3       |222688  |
|2017            |4       |23116   |
|2018            |1       |252     |
|2018            |2       |558329  |
|2018            |3       |281790  |
|2018            |4       |24539   |
|2019            |1       |196     |
|2019            |2       |666595  |
|2019            |3       |230686  |
|2019            |4       |26939   |
+----------------+--------+--------+

In [7]:
df = spark.read.csv(dataset_path_2, header=True)
# df.printSchema()
df = df.withColumn("Wind_Scale", get_wind_scale(F.col("Wind_Speed")))

column_name = "Wind_Scale"
wind_scale_values = ["calm", "light air", "light breeze", "gentle breeze", "moderate breeze", "fresh breeze", "strong breeze", "near gale", "gale", "severe gale", "storm", "violent storm", "hurricane"]
names = [as_column_name(column_name, v) for v in wind_scale_values]
for new_column_name in names:
    df = df.withColumn(new_column_name, assing_value(F.lit(column_name), F.lit(new_column_name), F.col(column_name)))
    
# df.filter(F.col("Wind_Speed") == F.col("Wind_Speed_pred")).select(*[F.col(c) for c in ["Wind_Speed", "Wind_Scale", 'wind_scale__calm', 'wind_scale__light_air', 'wind_scale__light_breeze', 'wind_scale__gentle_breeze', 'wind_scale__moderate_breeze', 'wind_scale__fresh_breeze', 'wind_scale__strong_breeze', 'wind_scale__near_gale', 'wind_scale__gale', 'wind_scale__severe_gale', 'wind_scale__storm', 'wind_scale__violent_storm', 'wind_scale__hurricane']]).show(10, False)
df = df.drop_duplicates() # 2855860 / 2881493
df.repartition(1).write.csv(output_path + "accidentes_dummies_windspeed_windscale", header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
df = spark.read.csv(output_path + "accidentes_dummies_windspeed_windscale", header=True)
df1 = df.groupBy("start_time__year", "Severity").agg(F.count("*").alias("total_count")).orderBy("start_time__year", "Severity")
df2 = df.groupBy("start_time__year").agg(F.count("*").alias("total_count")).orderBy("start_time__year")
df3 = (df1
    .join(df2, df1.start_time__year == df2.start_time__year, "inner")
    .select(df1.start_time__year, 
            df1.Severity, 
            F.concat_ws("-", df1.start_time__year, df1.Severity).alias("group"),
            df1.total_count, 
            df2.total_count,
            (F.round(df1.total_count / df2.total_count, 2)).alias("fraction"), 
            ((df1.total_count / df2.total_count)*100).alias("portion")
    )
)
df3.show()
for row in df3.collect():
    print("{},{},{},{},{},{},{}".format(row.start_time__year, row.Severity, row.group, row.total_count, row.total_count, row.fraction, row.portion))

fractions = {row.group: row.fraction for row in df3.collect()}
print(fractions)
df.withColumn("group", F.concat_ws("-", df1.start_time__year, df1.Severity)).sampleBy("group", fractions=fractions, seed=0).groupBy("start_time__year", "Severity").agg(F.count("*").alias("total_count")).orderBy("start_time__year", "Severity").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------+------+-----------+-----------+--------+--------------------+
|start_time__year|Severity| group|total_count|total_count|fraction|             portion|
+----------------+--------+------+-----------+-----------+--------+--------------------+
|            2016|       1|2016-1|        219|     397771|     0.0| 0.05505680404051577|
|            2016|       2|2016-2|     261987|     397771|    0.66|   65.86377589115345|
|            2016|       3|2016-3|     121901|     397771|    0.31|  30.646024974168558|
|            2016|       4|2016-4|      13664|     397771|    0.03|  3.4351423306374773|
|            2017|       1|2017-1|        270|     694396|     0.0| 0.03888271245802107|
|            2017|       2|2017-2|     448322|     694396|    0.65|   64.56287190594416|
|            2017|       3|2017-3|     222688|     694396|    0.32|   32.06930915500665|
|            2017|       4|2017-4|      23116|     694396|    0.03|   3.328936226591167|
|            2018|   